# 🚀 Neo4j Installation on Ubuntu

This guide helps you install **Neo4j Community Edition** on Ubuntu (20.04, 22.04, etc.)
*Also do refer https://neo4j.com/docs/operations-manual/current/installation/ if any issues with installation*

---

## ✅ Step 1: Install Java (required)

Neo4j requires Java 11 or later.

```bash
sudo apt update
sudo apt install openjdk-17-jdk -y
java -version

wget -O - https://debian.neo4j.com/neotechnology.gpg.key | sudo gpg --dearmor -o /usr/share/keyrings/neo4j.gpg

echo "deb [signed-by=/usr/share/keyrings/neo4j.gpg] https://debian.neo4j.com stable 5" | sudo tee /etc/apt/sources.list.d/neo4j.list

sudo apt update
sudo apt install neo4j -y

sudo systemctl enable neo4j
sudo systemctl start neo4j
```

## Access Neo4j Browser

Open your browser and go to:
http://localhost:7474

Default credentials:

    - Username: neo4j

    - Password: neo4j (you will be prompted to change it)
# Install Neo4j Package in Local
> pip install neo4j


In [ ]:
from neo4j import GraphDatabase
class Neo4JConnection:
    def __init__(self, uri, user, password):
        """Initialize the Neo4j connection with URI, username, and password."""
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        """Close the Neo4j connection."""
        if self.driver:
            self.driver.close()

    def execute_query(self, query, parameters=None):
        """Execute a Cypher query and return the result."""
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result.data()
# Dino v2 or grounded Dino

# Functions to perform action for creating, deleting and updating Knowledge graphs for a patient

In [ ]:
# Function for deleting all nodes
def delete_all_nodes(connection):
    query = """
    MATCH (n)
    DETACH DELETE n;
    """
    connection.execute_query(query)
    print("All nodes deleted.")
    
# Function for initialising the database with respiratory distress nodes and relationships
def initialise_respiratory_distress(connection): # Both nodes and relationships
    query = """
    CREATE
    (tp:Symptom {name: "Tripod position", value: "NA"}),
    (gasp:Symptom {name: "Open mouth with intermittent gasping sounds", value: "NA"}),
    (abn_head:Symptom {name: "Abnormal head or neck position", value: "NA"}),
    (rrate:Symptom {name: "Respiratory Rate", value: "NA"}), // could later be "under_8", "between_8_28", or "above_28"
    (cyanosis:Symptom {name: "Cyanosis visible on exposed skin", value: "NA"}),
    (unequal:Symptom {name: "Unequal chest-wall movement", value: "NA"}),
    (arrhythmic:Symptom {name: "Arrhythmic chest movement", value: "NA"}),

    (rrA:RRCase {name: "RR Human case A", value: "NA"}),
    (rrB:RRCase {name: "RR Human case B", value: "NA"}),
    (rrD:RRCase {name: "RR Human case D", value: "NA"}),

    (hrd:Condition {name: "Human Respiratory Distress", value: "NA"}),

    (rd:Diagnosis {name: "Respiratory Distress", value: "NA"}),
    (mrd:Diagnosis {name: "Manekin Respiratory Distress", value: "NA"}),
    
    // Relationships
    (tp)-[:INDICATES]->(rrA),
    (gasp)-[:INDICATES]->(rrA),
    (abn_head)-[:INDICATES]->(rrB),
    (gasp)-[:INDICATES]->(rrB),
    (rrate)-[:INDICATES]->(rrD),

    (rrA)-[:LEADS_TO]->(hrd),
    (rrB)-[:LEADS_TO]->(hrd),
    (rrD)-[:LEADS_TO]->(hrd),
    (cyanosis)-[:INDICATES]->(hrd),

    (unequal)-[:INDICATES]->(mrd),
    (arrhythmic)-[:INDICATES]->(mrd),
    
    (hrd)-[:CAUSES]->(rd),
    (mrd)-[:CAUSES]->(rd);
    """
    result = connection.execute_query(query)
    for record in result:
        print(record)

# Function for updating the initial respiratory distress nodes and relationships
def update_respiratory_distress(connection, data):
    rrA_value = "yes" if data["Tripod position"] == "yes" and data["Open mouth with intermittent gasping sounds"] == "yes" else "no"
    rrB_value = "yes" if data["Abnormal head or neck position"] == "yes" and data["Open mouth with intermittent gasping sounds"] == "yes" else "no"
    rrD_value = "yes" if data["Respiratory Rate"] != "between_8_28" else "no"
    hrd_value = "yes" if rrA_value == "yes" or rrB_value == "yes" or rrD_value == "yes" or data["Cyanosis visible on exposed skin"] == "yes" else "no"
    mrd_value = "yes" if data["Unequal chest-wall movement"] == "yes" or data["Arrhythmic chest movement"] == "yes" else "no"
    rd_value = "yes" if mrd_value == "yes" or hrd_value == "yes" else "no"
    
    queries = [
        ("Symptom", "Tripod position", data["Tripod position"]),
        ("Symptom", "Open mouth with intermittent gasping sounds", data["Open mouth with intermittent gasping sounds"]),
        ("Symptom", "Abnormal head or neck position", data["Abnormal head or neck position"]),
        ("Symptom", "Respiratory Rate", data["Respiratory Rate"]),
        ("Symptom", "Cyanosis visible on exposed skin", data["Cyanosis visible on exposed skin"]),
        ("Symptom", "Unequal chest-wall movement", data["Unequal chest-wall movement"]),
        ("Symptom", "Arrhythmic chest movement", data["Arrhythmic chest movement"]),
        ("RRCase", "RR Human case A", rrA_value),
        ("RRCase", "RR Human case B", rrB_value),
        ("RRCase", "RR Human case D", rrD_value),
        ("Condition", "Human Respiratory Distress", hrd_value),
        ("Diagnosis", "Manekin Respiratory Distress", mrd_value),
        ("Diagnosis", "Respiratory Distress", rd_value)
    ]

    with connection.driver.session() as session:
        for label, name, value in queries:
            cypher = f"""
            MATCH (n:{label} {{name: $name}})
            SET n.value = $value
            """
            session.run(cypher, name=name, value=value)

    print("✅ All symptom and reasoning nodes updated successfully.")

In [42]:
def initialise_trauma_logic(connection):
    query = """
    CREATE
    // Symptom nodes
    (occluded:Symptom {name: "Body region occluded", value: "NA"}),
    (verbal_amputation:Symptom {name: "Verbal confirmation of amputation", value: "NA"}),
    (removal:Symptom {name: "Removal of body part", value: "NA"}),
    (blood:Symptom {name: "Visible blood around injury", value: "NA"}),
    (damaged_clothing:Symptom {name: "Damaged clothing", value: "NA"}),
    (verbal_wound:Symptom {name: "Verbal confirmation of wound", value: "NA"}),
    (inaccessible:Symptom {name: "Otherwise inaccessible", value: "NA"}),
    (is_head:Symptom {name: "Is head?", value: "NA"}),
    (is_torso:Symptom {name: "Is torso?", value: "NA"}),
    (movement:Symptom {name: "Movement indicating injury", value: "NA"}),
    (inability:Symptom {name: "Inability to move", value: "NA"}),
    (deformity:Symptom {name: "Visible deformity of limbs", value: "NA"}),
    (burn:Symptom {name: "Burn", value: "NA"}),
    (abrasion:Symptom {name: "Abrasion", value: "NA"}),
    (hemorrhage:Symptom {name: "Hemorrhage", value: "NA"}),

    // Reasoning nodes (intermediate logic)
    (amputation_caseA:RRCase {name: "Trauma amputation case A", value: "NA"}),
    (is_extremity:RRCase {name: "Is extremity?", value: "NA"}),
    (wound_caseA:RRCase {name: "Trauma wound case A", value: "NA"}),
    (wound_caseB:RRCase {name: "Trauma wound case B", value: "NA"}),
    (wound_caseC:RRCase {name: "Trauma wound case C", value: "NA"}),
    (visible_wound:RRCase {name: "Visible wound", value: "NA"}),
    
    // Diagnostic/Condition nodes
    (not_testable:RRCase {name: "Trauma not testable", value: "NA"}),
    (trauma_amputation:Condition {name: "Trauma amputation", value: "NA"}),
    (trauma_wound:Condition {name: "Trauma wound", value: "NA"}),
    (trauma_normal:Condition {name: "Trauma normal", value: "NA"}),

    // Relationships
    // Traumma not testable
    (occluded)-[:INDICATES]->(not_testable),
    (inaccessible)-[:INDICATES]->(not_testable),

    // Trauma amputation
    (is_head)-[:INDICATES]->(is_extremity),
    (is_torso)-[:INDICATES]->(is_extremity),
    (is_extremity)-[:INDICATES]->(trauma_amputation),
    (verbal_amputation)-[:INDICATES]->(trauma_amputation),
    (removal)-[:INDICATES]->(amputation_caseA),
    (blood)-[:INDICATES]->(amputation_caseA),
    (amputation_caseA)-[:LEADS_TO]->(trauma_amputation),
    (not_testable)-[:LEADS_TO]->(trauma_amputation),
    
    (movement)-[:INDICATES]->(wound_caseB),
    (inability)-[:INDICATES]->(wound_caseB),
    (is_extremity)-[:INDICATES]->(wound_caseB),
    (wound_caseB)-[:LEADS_TO]->(trauma_wound),
    (deformity)-[:INDICATES]->(wound_caseC),
    (wound_caseC)-[:LEADS_TO]->(trauma_wound),
    (blood)-[:INDICATES]->(wound_caseA),
    (damaged_clothing)-[:INDICATES]->(wound_caseA),
    (verbal_wound)-[:INDICATES]->(trauma_wound),
    (wound_caseA)-[:LEADS_TO]->(trauma_wound),
    (trauma_amputation)-[:LEADS_TO]->(trauma_wound),
    (visible_wound)-[:LEADS_TO]->(wound_caseC),
    (abrasion)-[:INDICATES]->(visible_wound),
    (burn)-[:INDICATES]->(visible_wound),
    (hemorrhage)-[:INDICATES]->(visible_wound),

    (trauma_wound)-[:CAUSES]->(trauma_normal);
    """
    result = connection.execute_query(query)
    print("✅ Trauma symptom and reasoning graph initialized.")


In [43]:
def update_trauma_logic(connection, data):
    # Derived logic values
    is_extremity_value = "yes" if data["Is head?"] == "yes" or data["Is torso?"] == "yes" else "no"
    trauma_not_testable_value = "yes" if data["Body region occluded"] == "yes" or data["Otherwise inaccessible"] == "yes" else "no"
    amputation_caseA_value = "yes" if data["Removal of body part"] == "yes" or data["Visible blood around injury"] == "yes" else "no"
    
    trauma_amputation_value = "yes" if (
        is_extremity_value == "yes" or
        data["Verbal confirmation of amputation"] == "yes" or
        amputation_caseA_value == "yes" or
        trauma_not_testable_value == "yes"
    ) else "no"

    wound_caseA_value = "yes" if (
        data["Visible blood around injury"] == "yes" or
        data["Damaged clothing"] == "yes"
    ) else "no"

    wound_caseB_value = "yes" if (
        is_extremity_value == "yes" or
        data["Movement indicating injury"] == "yes" or
        data["Inability to move"] == "yes"
    ) else "no"

    visible_wound_value = "yes" if (
        data["Burn"] == "yes" or
        data["Abrasion"] == "yes" or
        data["Hemorrhage"] == "yes"
    ) else "no"

    wound_caseC_value = "yes" if (
        data["Visible deformity of limbs"] == "yes" or
        visible_wound_value == "yes"
    ) else "no"

    trauma_wound_value = "yes" if (
        data["Verbal confirmation of wound"] == "yes" or
        wound_caseA_value == "yes" or
        wound_caseB_value == "yes" or
        wound_caseC_value == "yes" or
        trauma_amputation_value == "yes"
    ) else "no"

    trauma_normal_value = "yes" if trauma_wound_value == "yes" else "no"

    queries = [
        ("Symptom", "Body region occluded", data["Body region occluded"]),
        ("Symptom", "Otherwise inaccessible", data["Otherwise inaccessible"]),
        ("Symptom", "Verbal confirmation of amputation", data["Verbal confirmation of amputation"]),
        ("Symptom", "Removal of body part", data["Removal of body part"]),
        ("Symptom", "Visible blood around injury", data["Visible blood around injury"]),
        ("Symptom", "Damaged clothing", data["Damaged clothing"]),
        ("Symptom", "Verbal confirmation of wound", data["Verbal confirmation of wound"]),
        ("Symptom", "Is head?", data["Is head?"]),
        ("Symptom", "Is torso?", data["Is torso?"]),
        ("Symptom", "Movement indicating injury", data["Movement indicating injury"]),
        ("Symptom", "Inability to move", data["Inability to move"]),
        ("Symptom", "Visible deformity of limbs", data["Visible deformity of limbs"]),
        ("Symptom", "Burn", data["Burn"]),
        ("Symptom", "Abrasion", data["Abrasion"]),
        ("Symptom", "Hemorrhage", data["Hemorrhage"]),
        
        ("RRCase", "Is extremity?", is_extremity_value),
        ("RRCase", "Trauma amputation case A", amputation_caseA_value),
        ("RRCase", "Trauma wound case A", wound_caseA_value),
        ("RRCase", "Trauma wound case B", wound_caseB_value),
        ("RRCase", "Trauma wound case C", wound_caseC_value),
        ("RRCase", "Visible wound", visible_wound_value),
        ("RRCase", "Trauma not testable", "yes" if data["Body region occluded"] == "yes" or data["Otherwise inaccessible"] == "yes" else "no"),

        ("Condition", "Trauma amputation", trauma_amputation_value),
        ("Condition", "Trauma wound", trauma_wound_value),
        ("Condition", "Trauma normal", trauma_normal_value)
    ]

    with connection.driver.session() as session:
        for label, name, value in queries:
            cypher = f"""
            MATCH (n:{label} {{name: $name}})
            SET n.value = $value
            """
            session.run(cypher, name=name, value=value)

    print("✅ Trauma symptom and reasoning nodes updated successfully.")


In [44]:
def initialise_eye_logic(connection):
    query = """
    CREATE
    // Symptom nodes
    (injured:Symptom {name: "Injured eyelid", value: "NA"}),
    (occluded:Symptom {name: "Occluded eyelid", value: "NA"}),
    (both_closed:Symptom {name: "Both eyelids closed", value: "NA"}),
    (responsive:Symptom {name: "Responsive to prompts", value: "NA"}),
    (eyes_blinking:Symptom {name: "Eyes open and blinking", value: "NA"}),
    (both_open_still:Symptom {name: "Both eyelids open without movement", value: "NA"}),

    // Reasoning nodes
    (closed_caseA:RRCase {name: "Closed case A", value: "NA"}),

    // Condition / logic outcomes
    (not_testable:Condition {name: "Not testable", value: "NA"}),
    (closed:Condition {name: "Closed", value: "NA"}),
    (open:Condition {name: "Open", value: "NA"}),
    (open_human:Condition {name: "Open for human", value: "NA"}),
    (open_manekin:Condition {name: "Open for manekin", value: "NA"}),

    // Relationships
    (injured)-[:INDICATES]->(not_testable),
    (occluded)-[:INDICATES]->(not_testable),

    (both_closed)-[:INDICATES]->(closed_caseA),
    (responsive)-[:INDICATES]->(closed_caseA),
    (closed_caseA)-[:LEADS_TO]->(closed),
    (not_testable)-[:LEADS_TO]->(closed),

    (closed)-[:LEADS_TO]->(open),
    (eyes_blinking)-[:INDICATES]->(open_human),
    (open_human)-[:LEADS_TO]->(open),

    (both_open_still)-[:INDICATES]->(open_manekin),
    (open_manekin)-[:LEADS_TO]->(open);
    """
    connection.execute_query(query)
    print("✅ Eye symptom and reasoning graph initialized.")
def update_eye_logic(connection, data):
    closed_caseA_value = "yes" if data["Both eyelids closed"] == "yes" or data["Responsive to prompts"] == "yes" else "no"
    not_testable_value = "yes" if data["Injured eyelid"] == "yes" or data["Occluded eyelid"] == "yes" else "no"
    closed_value = "yes" if closed_caseA_value == "yes" or not_testable_value == "yes" else "no"
    open_human_value = "yes" if data["Eyes open and blinking"] == "yes" else "no"
    open_manekin_value = "yes" if data["Both eyelids open without movement"] == "yes" else "no"
    open_value = "yes" if open_human_value == "yes" or open_manekin_value == "yes" else "no"

    queries = [
        ("Symptom", "Injured eyelid", data["Injured eyelid"]),
        ("Symptom", "Occluded eyelid", data["Occluded eyelid"]),
        ("Symptom", "Both eyelids closed", data["Both eyelids closed"]),
        ("Symptom", "Responsive to prompts", data["Responsive to prompts"]),
        ("Symptom", "Eyes open and blinking", data["Eyes open and blinking"]),
        ("Symptom", "Both eyelids open without movement", data["Both eyelids open without movement"]),

        ("RRCase", "Closed case A", closed_caseA_value),

        ("Condition", "Not testable", not_testable_value),
        ("Condition", "Closed", closed_value),
        ("Condition", "Open for human", open_human_value),
        ("Condition", "Open for manekin", open_manekin_value),
        ("Condition", "Open", open_value)
    ]

    with connection.driver.session() as session:
        for label, name, value in queries:
            cypher = f"""
            MATCH (n:{label} {{name: $name}})
            SET n.value = $value
            """
            session.run(cypher, name=name, value=value)

    print("✅ Eye assessment nodes updated successfully.")


In [45]:
def initialise_alertness_logic(connection):
    query = """
    CREATE
    // Condition nodes
    (rr:Condition {name: "Respiratory Rate", value: "NA"}),
    (trauma:Condition {name: "Trauma", value: "NA"}),

    // Decision node
    (is_manekin:RRCase {name: "Is manekin?", value: "NA"}),

    // Outcome condition nodes
    (alert_verbal:Condition {name: "Alertness: Verbal", value: "NA"}),
    (alert_ocular:Condition {name: "Alertness: Ocular", value: "NA"}),
    (alert_motor:Condition {name: "Alertness: Motor", value: "NA"}),

    // Relationships
    (rr)-[:INDICATES]->(is_manekin),
    (trauma)-[:INDICATES]->(is_manekin),
    (is_manekin)-[:LEADS_TO]->(alert_verbal),
    (is_manekin)-[:LEADS_TO]->(alert_ocular),
    (is_manekin)-[:LEADS_TO]->(alert_motor);
    """
    connection.execute_query(query)
    print("✅ Alertness logic graph initialized.")
def update_alertness_logic(connection, data):
    queries = [
        ("Condition", "Respiratory Rate", data["Respiratory Rate"]),
        ("Condition", "Trauma", data["Trauma"]),
        ("RRCase", "Is manekin?", data["Is manekin?"]),
        ("Condition", "Alertness: Verbal", data["Alertness: Verbal"]),
        ("Condition", "Alertness: Ocular", data["Alertness: Ocular"]),
        ("Condition", "Alertness: Motor", data["Alertness: Motor"])
    ]

    with connection.driver.session() as session:
        for label, name, value in queries:
            cypher = f"""
            MATCH (n:{label} {{name: $name}})
            SET n.value = $value
            """
            session.run(cypher, name=name, value=value)

    print("✅ Alertness assessment nodes updated successfully.")


In [ ]:
def add_observation_with_logic(
    connection,
    patient_id,
    name,
    dob,
    observation_id,
    timestamp,
    location,
    vitals,
    trauma,
    logic_inputs_dicts  # e.g., {"resp": data_rr, "trauma": data_trauma, ...}
):
    # Step 1: Create patient & observation
    trauma_status_dict = trauma  # same format
    obs_query = f"""
    MERGE (P:Patient {{id: $patient_id}})
    ON CREATE SET P.name = $name, P.dob = $dob

    CREATE (obs:Observation {{
        observation_id: $observation_id,
        timestamp: datetime($timestamp),
        observation_type: "Ground Robots",
        robot_id: "A7563"
    }})

    CREATE (loc:Location {{
        loc_id: $observation_id + "-loc",
        latitude: $latitude,
        longitude: $longitude
    }})

    CREATE (hr:HeartRate {{hr_id: $observation_id + "-hr", value: $heart_rate, unit: "BPM"}})
    CREATE (rr:RespiratoryRate {{rr_id: $observation_id + "-rr", value: $respiratory_rate, unit: "BrPM"}})
    CREATE (temp:Temperature {{temp_id: $observation_id + "-temp", value: $temperature, unit: "F"}})

    MERGE (P)-[:HAS_OBSERVATION]->(obs)
    CREATE (obs)-[:RECORDED_AT]->(loc)
    CREATE (obs)-[:HAS_VITAL]->(hr)
    CREATE (obs)-[:HAS_VITAL]->(rr)
    CREATE (obs)-[:HAS_VITAL]->(temp)
    """

    # Add trauma nodes and link
    for part, status in trauma_status_dict.items():
        safe_var = part.replace(" ", "_").replace(".", "").replace("-", "")
    obs_query += f"""
    CREATE ({safe_var}:Trauma {{body_part: "{part}", status: "{status}"}})
    CREATE (obs)-[:HAS_TRAUMA]->({safe_var})
    """

    # Add temporal chaining
    obs_query += """
    WITH P, obs, datetime($timestamp) AS new_obs_time
    OPTIONAL MATCH (P)-[:HAS_OBSERVATION]->(prev_obs)
    WHERE prev_obs.timestamp < new_obs_time
    WITH P, obs, prev_obs
    ORDER BY prev_obs.timestamp DESC
    LIMIT 1
    FOREACH (_ IN CASE WHEN prev_obs IS NOT NULL THEN [1] ELSE [] END |
      CREATE (prev_obs)-[:NEXT_OBSERVATION]->(obs)
    )
    """

    parameters = {
        "patient_id": patient_id,
        "name": name,
        "dob": dob,
        "observation_id": observation_id,
        "timestamp": timestamp,
        "latitude": location[0],
        "longitude": location[1],
        "heart_rate": vitals["hr"],
        "respiratory_rate": vitals["rr"],
        "temperature": vitals["temp"]
    }

    connection.execute_query(obs_query, parameters)

    # Step 2: Call domain-specific logic updates and link each to this observation
    if "resp" in logic_inputs_dicts:
        update_respiratory_distress(connection, logic_inputs_dicts["resp"])
        _link_logic_nodes_to_obs(connection, observation_id, "Respiratory")

    if "trauma" in logic_inputs_dicts:
        update_trauma_logic(connection, logic_inputs_dicts["trauma"])
        _link_logic_nodes_to_obs(connection, observation_id, "Trauma")

    if "eye" in logic_inputs_dicts:
        update_eye_logic(connection, logic_inputs_dicts["eye"])
        _link_logic_nodes_to_obs(connection, observation_id, "Eye")

    if "alertness" in logic_inputs_dicts:
        update_alertness_logic(connection, logic_inputs_dicts["alertness"])
        _link_logic_nodes_to_obs(connection, observation_id, "Alertness")

    print(f"✅ Created observation {observation_id} and applied logic modules.")


def _link_logic_nodes_to_obs(connection, observation_id, logic_scope):
    """
    Helper: for every node with a value ≠ 'NA' in the given logic scope, link to the Observation
    """
    query = f"""
    MATCH (obs:Observation {{observation_id: $obs_id}})
    MATCH (n)
    WHERE n.value <> "NA" AND (
        (n:Symptom OR n:RRCase OR n:Condition OR n:Diagnosis)
    )
    MERGE (obs)-[:HAS_LOGIC]->(n)
    """
    connection.execute_query(query, {"obs_id": observation_id})


In [47]:
data_rr = {
    "Tripod position": "yes",
    "Open mouth with intermittent gasping sounds": "yes",
    "Abnormal head or neck position": "no",
    "Respiratory Rate": "above_28",
    "Cyanosis visible on exposed skin": "no",
    "Unequal chest-wall movement": "yes",
    "Arrhythmic chest movement": "no"
}
data_trauma = {
    "Body region occluded": "no",
    "Otherwise inaccessible": "no",
    "Verbal confirmation of amputation": "yes",
    "Removal of body part": "yes",
    "Visible blood around injury": "yes",
    "Damaged clothing": "yes",
    "Verbal confirmation of wound": "no",
    "Is head?": "no",
    "Is torso?": "yes",
    "Is extremity?": "yes",
    "Movement indicating injury": "no",
    "Inability to move": "yes",
    "Visible deformity of limbs": "no",
    "Burn": "no",
    "Abrasion": "yes",
    "Hemorrhage": "no"
}
data_eye = {
    "Injured eyelid": "no",
    "Occluded eyelid": "no",
    "Both eyelids closed": "yes",
    "Responsive to prompts": "yes",
    "Eyes open and blinking": "no",
    "Both eyelids open without movement": "no"
}
data_alertness = {
    "Respiratory Rate": "above_28",
    "Trauma": "yes",
    "Is manekin?": "no",
    "Alertness: Verbal": "yes",
    "Alertness: Ocular": "yes",
    "Alertness: Motor": "no"
}


In [17]:

uri = "bolt://localhost:7687"  # Default Bolt URI for local Neo4j
user = "neo4j"              # Default username
password = "ARzY@RM$7GpCt@9"

# Initialize Neo4j connection
neo4j_conn = Neo4JConnection(uri, user, password)


## Expected Structure of a KG as per Darpa Triage rules(following Baseyian network architecture by Szymon Rusiecki)

In [62]:
def create_KG_for_a_patient():
    delete_all_nodes(neo4j_conn)
    initialise_respiratory_distress(neo4j_conn)
    initialise_eye_logic(neo4j_conn)
    initialise_trauma_logic(neo4j_conn)
    initialise_alertness_logic(neo4j_conn)
    
    
    update_respiratory_distress(neo4j_conn, data_rr)
    update_trauma_logic(neo4j_conn, data_trauma)
    update_eye_logic(neo4j_conn, data_eye)
    update_alertness_logic(neo4j_conn, data_alertness)
create_KG_for_a_patient()


/tmp/ipykernel_34217/184350347.py:14: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:
/tmp/ipykernel_34217/3950034971.py:78: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with connection.driver.session() as session:
/tmp/ipykernel_34217/606285847.py:76: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with connection.driver.session() as session:


All nodes deleted.
✅ Eye symptom and reasoning graph initialized.
✅ Trauma symptom and reasoning graph initialized.
✅ Alertness logic graph initialized.
✅ All symptom and reasoning nodes updated successfully.
✅ Trauma symptom and reasoning nodes updated successfully.
✅ Eye assessment nodes updated successfully.
✅ Alertness assessment nodes updated successfully.


/tmp/ipykernel_34217/2243434324.py:65: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with connection.driver.session() as session:
/tmp/ipykernel_34217/3409384287.py:35: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with connection.driver.session() as session:


## Now paste below code in Neo4j browser and run

> MATCH (n)
OPTIONAL MATCH (n)-[r]->(m)
RETURN n, r, m;

In [ ]:
neo4j_conn.close()
# Run to close the connection

# Further work is integrating this KG into one for each observation recorded for a patient.

##  Expected structure is given by below code

In [ ]:
from time import time

In [ ]:
def patient_1(connection):
    query = """
    CREATE (
    // Create Nodes
       P1:Patient {id: "P001", name: "John Doe", dob: "1980-01-01"}),
       (obs1:Observation {observation_id: "P001-OBS001",timestamp: datetime("2025-04-08T10:15:00"),observation_type: "Ground Robots", robot_id: "A7563"}),
       (loc1:Location {loc_id:"P001-OBS001-l1", latitude: 40.4433, longitude: -79.9436}),
       (hr1:HeartRate {hr_id:"P001-OBS001-hr1", value: 105, unit: "BPM"}),
       (rr1:RespiratoryRate {rr_id:"P001-OBS001-rr1", value: 30, unit: "BrPM"}),
       (temp1:Temperature {temp_id:"P001-OBS001-temp1",value: 101.2, unit: "F"}),
       (hemorrhage1:SevereHemorrhage {status: "Present"}),
       (resp_distress1:RespiratoryDistress {status: "Present"}),
       (head1:Trauma {body_part: "Head", status: "Wound"}),
       (torso1:Trauma {body_part: "Torso", status: "Normal"}),
       (upper_ext1:Trauma {body_part: "Upper Ext.", status: "Wound"}),
       (lower_ext1:Trauma {body_part: "Lower Ext.", status: "Amputation"}),
    // Connect nodes
       (P1)-[:HAS_OBSERVATION]->(obs1)-[:RECORDED_AT]->(loc1),
       (obs1)-[:HAS_VITAL]->(hr1),
       (obs1)-[:HAS_VITAL]->(rr1),
       (obs1)-[:HAS_VITAL]->(temp1),
       (obs1)-[:HAS_Critical]->(hemorrhage1),
       (obs1)-[:HAS_Critical]->(resp_distress1),
       (obs1)-[:HAS_TRAUMA]->(head1),
       (obs1)-[:HAS_TRAUMA]->(torso1),
       (obs1)-[:HAS_TRAUMA]->(upper_ext1),
       (obs1)-[:HAS_TRAUMA]->(lower_ext1);
    """
    connection.execute_query(query)
    print("✅ graph initialized.")
    
def add_to_existing_graph_first(connection):
    query = """
    MATCH (P1:Patient {id: "P001"})
    CREATE
    // Create Nodes
        (obs2:Observation {observation_id: "P001-OBS002",timestamp: datetime("2025-04-08T11:00:00"),observation_type: "Ground Robots", robot_id: "A7563"}),
        (loc2:Location {loc_id:"P001-OBS002-l2", latitude: 42.00, longitude: -75.00}),
        (hr2:HeartRate {hr_id:"P001-OBS001-hr2", value: 98, unit: "BPM"}),
        (rr2:RespiratoryRate {rr_id:"P001-OBS002-rr2", value: 20, unit: "BrPM"}),
        (temp2:Temperature {temp_id:"P001-OBS002-temp2",value: 85, unit: "F"}),
        (hemorrhage2:SevereHemorrhage {status: "Present"}),
        (resp_distress2:RespiratoryDistress {status: "Present"}),
        (head2:Trauma {body_part: "Head", status: "Wound"}),
        (torso2:Trauma {body_part: "Torso", status: "Normal"}),
        (upper_ext2:Trauma {body_part: "Upper Ext.", status: "Wound"}),
        (lower_ext2:Trauma {body_part: "Lower Ext.", status: "Amputation"}),
    // Connect nodes
        (P1)-[:HAS_OBSERVATION]->(obs2)-[:RECORDED_AT]->(loc2),
        (obs2)-[:HAS_VITAL]->(hr2),
        (obs2)-[:HAS_VITAL]->(rr2),
        (obs2)-[:HAS_VITAL]->(temp2),
        (obs2)-[:HAS_Critical]->(hemorrhage2),
        (obs2)-[:HAS_Critical]->(resp_distress2),
        (obs2)-[:HAS_TRAUMA]->(head2),
        (obs2)-[:HAS_TRAUMA]->(torso2),
        (obs2)-[:HAS_TRAUMA]->(upper_ext2),
        (obs2)-[:HAS_TRAUMA]->(lower_ext2)
    WITH P1, obs2, obs2.timestamp AS new_obs_time
    OPTIONAL MATCH (P1)-[:HAS_OBSERVATION]->(obs1)
    WHERE obs1.timestamp < new_obs_time
    WITH P1, obs2, obs1
    ORDER BY obs1.timestamp DESC
    LIMIT 1
    FOREACH (_ IN CASE WHEN obs1 IS NOT NULL THEN [1] ELSE [] END |
    CREATE (obs1)-[:NEXT_OBSERVATION]->(obs2)
    );
    """
    connection.execute_query(query)
    print("✅ graph augmented once.")
    
def add_to_existing_graph_second(connection):
    query = """
    MATCH (P1:Patient {id: "P001"})
    CREATE
    // Create Nodes
        (obs4:Observation {observation_id: "P001-OBS003",timestamp: datetime("2025-04-08T11:20:00"),observation_type: "Ground Robots", robot_id: "A7563"}),
        (loc4:Location {loc_id:"P001-OBS003-l3", latitude: 42.00, longitude: -75.00}),
        (hr4:HeartRate {hr_id:"P001-OBS003-hr3", value: 20, unit: "BPM"}),
        (rr4:RespiratoryRate {rr_id:"P001-OBS003-rr3", value: 5, unit: "BrPM"}),
        (temp4:Temperature {temp_id:"P001-OBS003-temp3",value: 30, unit: "F"}),
        (hemorrhage4:SevereHemorrhage {status: "Present"}),
        (resp_distress4:RespiratoryDistress {status: "Present"}),
        (head4:Trauma {body_part: "Head", status: "Not Testable"}),
        (torso4:Trauma {body_part: "Torso", status: "Not Testable"}),
        (upper_ext4:Trauma {body_part: "Upper Ext.", status: "Not Testable"}),
        (lower_ext4:Trauma {body_part: "Lower Ext.", status: "Not Testable"}),
    // Connect nodes
        (P1)-[:HAS_OBSERVATION]->(obs4)-[:RECORDED_AT]->(loc4),
        (obs4)-[:HAS_VITAL]->(hr4),
        (obs4)-[:HAS_VITAL]->(rr4),
        (obs4)-[:HAS_VITAL]->(temp4),
        (obs4)-[:HAS_Critical]->(hemorrhage4),
        (obs4)-[:HAS_Critical]->(resp_distress4),
        (obs4)-[:HAS_TRAUMA]->(head4),
        (obs4)-[:HAS_TRAUMA]->(torso4),
        (obs4)-[:HAS_TRAUMA]->(upper_ext4),
        (obs4)-[:HAS_TRAUMA]->(lower_ext4)
    WITH P1, obs4, obs4.timestamp AS new_obs_time
    OPTIONAL MATCH (P1)-[:HAS_OBSERVATION]->(obs2)
    WHERE obs2.timestamp < new_obs_time
    WITH P1, obs2, obs4
    ORDER BY obs2.timestamp DESC
    LIMIT 1
    FOREACH (_ IN CASE WHEN obs2 IS NOT NULL THEN [1] ELSE [] END |
    CREATE (obs2)-[:NEXT_OBSERVATION]->(obs4)
    );
    """
    connection.execute_query(query)
    print("✅ graph augmented second time.")

delete_all_nodes(neo4j_conn)
patient_1(neo4j_conn)
add_to_existing_graph_first(neo4j_conn)
add_to_existing_graph_second(neo4j_conn)


/tmp/ipykernel_34217/184350347.py:14: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


All nodes deleted.
✅ graph initialized.
✅ graph augmented once.
✅ graph augmented second time.


In [61]:
neo4j_conn.close()
# Run to close the connection

## Now paste below code in Neo4j browser and run

> MATCH (n)
OPTIONAL MATCH (n)-[r]->(m)
RETURN n, r, m;


# Future direction includes way to integrate this into one process and perform adaptive RAG policy to pass a sub-KG to VLM to augment its output for Triage purposes.

In [ ]:
add_observation_with_logic(
    connection=neo4j_conn,
    patient_id="P001",
    name="John Doe",
    dob="1980-01-01",
    observation_id="P001-OBS005",
    timestamp="2025-04-08T13:00:00",
    location=(40.44, -79.94),
    vitals={"hr": 90, "rr": 16, "temp": 98.7},
    trauma={"Head": "Wound", "Torso": "Normal", "Upper Ext.": "Wound", "Lower Ext.": "Normal"},
    logic_inputs_dicts={neo4j_conn.close()
# Run to close the connection
        "resp": data_rr,
        "trauma": data_trauma,
        "eye": data_eye,
        "alertness": data_alertness
    }
)


✅ All symptom and reasoning nodes updated successfully.
✅ Trauma symptom and reasoning nodes updated successfully.
✅ Eye assessment nodes updated successfully.
✅ Alertness assessment nodes updated successfully.
✅ Created observation P001-OBS005 and applied logic modules.


In [57]:
# New logic input values
data_rr_new = {
    "Tripod position": "no",
    "Open mouth with intermittent gasping sounds": "yes",
    "Abnormal head or neck position": "yes",
    "Respiratory Rate": "under_8",
    "Cyanosis visible on exposed skin": "yes",
    "Unequal chest-wall movement": "no",
    "Arrhythmic chest movement": "yes"
}

data_trauma_new = {
    "Body region occluded": "no",
    "Otherwise inaccessible": "no",
    "Verbal confirmation of amputation": "no",
    "Removal of body part": "no",
    "Visible blood around injury": "yes",
    "Damaged clothing": "yes",
    "Verbal confirmation of wound": "yes",
    "Is head?": "yes",
    "Is torso?": "no",
    "Is extremity?": "yes",
    "Movement indicating injury": "no",
    "Inability to move": "yes",
    "Visible deformity of limbs": "no",
    "Burn": "no",
    "Abrasion": "yes",
    "Hemorrhage": "yes"
}

data_eye_new = {
    "Injured eyelid": "no",
    "Occluded eyelid": "no",
    "Both eyelids closed": "no",
    "Responsive to prompts": "yes",
    "Eyes open and blinking": "yes",
    "Both eyelids open without movement": "no"
}

data_alertness_new = {
    "Respiratory Rate": "under_8",
    "Trauma": "yes",
    "Is manekin?": "no",
    "Alertness: Verbal": "no",
    "Alertness: Ocular": "yes",
    "Alertness: Motor": "yes"
}

# Add new observation at later timestamp
add_observation_with_logic(
    connection=neo4j_conn,
    patient_id="P001",
    name="John Doe",  # same name and dob reused
    dob="1980-01-01",
    observation_id="P001-OBS006",
    timestamp="2025-04-08T14:00:00",
    location=(40.4438, -79.9445),
    vitals={"hr": 95, "rr": 7, "temp": 97.5},
    trauma={"Head": "Wound", "Torso": "Normal", "Upper Ext.": "Normal", "Lower Ext.": "Wound"},
    logic_inputs_dicts={
        "resp": data_rr_new,
        "trauma": data_trauma_new,
        "eye": data_eye_new,
        "alertness": data_alertness_new
    }
)


✅ All symptom and reasoning nodes updated successfully.
✅ Trauma symptom and reasoning nodes updated successfully.
✅ Eye assessment nodes updated successfully.
✅ Alertness assessment nodes updated successfully.
✅ Created observation P001-OBS006 and applied logic modules.
